## Target- urgent withdrawn by nurse(within 24hr)
* shift_applications status == 'cancelled' + shift_application prevStatus == 'confirmed'


### IDs -> only look at shifts application, leave each shifts after
* shift_applications id
* shift_applications user_id
* shift_applications shift_id


### Numeric Field
* shifts rate
* shifts net_pay: rate - nursedash profit
* shift_applications distance
* shift break time
* user withdrawn times: 
    Problem: 
        1. We don't know in reality that you will withdraw your previous one.
        2. cannot be used to predict new user
* user applied times:
* prev SA/CW rate


## Time
### shift, shift app time
* S_Create (shift created) => SA_Create (application created) => CWTime (comfim withdrawn) => Start_Time (shift start)
* S_Create: shift create time
* SA_Create: shift application create time
* CWTime: shift withdrawn time, whole confirmed withdrawn population (include within 24hr and > 24hr)
* Start_Time: shift start time
##### calculated field, graph in hours
* S_Create2SA_Create: shift application create time - shift create time
* S_Create2Start_Time: shift start time - shift create time
        1. has negative values. delete maybe?
* SA_Create2Start_Time: shift start time - shift application create time
        1. has negative values. delete maybe?

### User
* user approvedAt:
* user createdAt:
#### Calculated field
* U_Create2U_approved: User approve time - User create time
* U_Create2now: now - User create time, in months(seconds/2629746)
* U_Approve2now: now - User approve time


### Categorical
* shifts role: position name + type
* facilities name = facilities short_name
* facilities areaId = facilities areaName
* withdrawnInfo_key, withdrawnInfo_value
* facilities segmentName: Senior Living = 1, Healthcare = 0
* Users enableNotifications:
* Users emailNotifications:
* Users appNotifications:
* Users allowedNotifications:


### ?
* facilities allowedQualification
* facilities createdAt
* facilities rates
* nurse shift withdrawn by admin, when policy start, how many withdrawn by admin
* shifts = unit
* role
* users updateAt ?
* users relationToFacility

# Connect to DB, Fetch data

In [2]:
import pandas as pd
# specify connection to database
import psycopg2
connection = psycopg2.connect(
    host="nursedash-prod.cuzi2kducsnv.us-east-1.rds.amazonaws.com",
    database="nursedash",
    user="external_analyst",
    password="uDps8APganhSLc3K2xe7NtMPq")

### <font color = green> all time to chicago time, No withdrawn info

In [3]:
df = pd.read_sql_query("""

SELECT  sa.id, sa.user_id, sa.shift_id, f.id AS facility_id, sa."withdrawnInfo" -> 'initiator' as withdrawnInfo_value,
sa."status", sa."prevStatus", sa."distance", "s"."description" AS "shift_description",
"s"."assigned_nurse_id", s."net_pay", "s"."unit" AS "s_unit",s."type",
"s"."qualifications" AS "s_qualifications", "s"."breakTime" AS "s_breakTime", sa."withdrawnInfo",
"f"."name" AS "f_name","f"."short_name" AS "f_short_name", f."segmentName", f."areaName",
timezone('America/Chicago', s."createdAt") as s_create,
timezone('America/Chicago', sa."createdAt") as sa_create,
timezone('America/Chicago', u."approvedAt") as u_approve,
timezone('America/Chicago', u."createdAt") as u_create,
timezone('America/Chicago', sa."statusUpdatedAt") as sa_statusUpdate,
timezone('America/Chicago', timezone('UTC', s.start_time)) AS "Start_Time" 
FROM shifts s
INNER JOIN shift_applications sa ON s.id = sa.shift_id
INNER JOIN facilities f ON s.facility_id = f.id
INNER JOIN users u ON sa.user_id = u.id

""", con = connection)

In [4]:
df.columns

Index(['id', 'user_id', 'shift_id', 'facility_id', 'withdrawninfo_value',
       'status', 'prevStatus', 'distance', 'shift_description',
       'assigned_nurse_id', 'net_pay', 's_unit', 'type', 's_qualifications',
       's_breakTime', 'withdrawnInfo', 'f_name', 'f_short_name', 'segmentName',
       'areaName', 's_create', 'sa_create', 'u_approve', 'u_create',
       'sa_statusupdate', 'Start_Time'],
      dtype='object')

### <font color = green> these features was done in sql before

### U_create2U_approve, in hours

In [5]:
from datetime import datetime
df['U_create2U_approve'] = df.apply(lambda row: (row['u_approve'] - row['u_create']).total_seconds()/3600, 
                                    axis = 1)

### U_create2now, in month

In [6]:
df['U_create2now'] = df.apply(lambda row: (datetime.now() - row['u_create']).total_seconds()/2629746, 
                              axis = 1)

### U_approve2now, in month

In [7]:
df['U_approve2now'] = df.apply(lambda row: (datetime.now() - row['u_approve']).total_seconds()/2629746, 
                               axis = 1)

### S_create2SA_Create, in hours

In [8]:
df['S_create2SA_Create'] = df.apply(lambda row: (row['sa_create'] - row['s_create']).total_seconds()/3600, 
                                    axis = 1)

### SA_Create2Start_Time

In [9]:
# some start time is at year 0019 
df['SA_Create2Start_Time'] = df.apply(lambda row: (row['Start_Time'] - row['sa_create']).total_seconds()/3600 if str(row['Start_Time'])[:2] != '00' else 9999, 
                                         axis = 1)

### S_Create2Start_Time

In [10]:
df['S_Create2Start_Time'] = df.apply(lambda row: (row['Start_Time'] - row['s_create']).total_seconds()/3600 if str(row['Start_Time'])[:2] != '00' else 9999, 
                                         axis = 1)

### CW_Time2Start_Time

In [11]:
df['CW_Time2Start_Time'] = df.apply(lambda row: (row['Start_Time'] - row['sa_statusupdate']).total_seconds()/3600 if str(row['Start_Time'])[:2] != '00' else 9999, 
                                         axis = 1)

# Data Cleaning

In [12]:
df.columns

Index(['id', 'user_id', 'shift_id', 'facility_id', 'withdrawninfo_value',
       'status', 'prevStatus', 'distance', 'shift_description',
       'assigned_nurse_id', 'net_pay', 's_unit', 'type', 's_qualifications',
       's_breakTime', 'withdrawnInfo', 'f_name', 'f_short_name', 'segmentName',
       'areaName', 's_create', 'sa_create', 'u_approve', 'u_create',
       'sa_statusupdate', 'Start_Time', 'U_create2U_approve', 'U_create2now',
       'U_approve2now', 'S_create2SA_Create', 'SA_Create2Start_Time',
       'S_Create2Start_Time', 'CW_Time2Start_Time'],
      dtype='object')

In [13]:
# There are some negative numbers in these column. Target all = 0, drop for now
df = df[(df["S_Create2Start_Time"]>0) & (df["SA_Create2Start_Time"]>0)]

# sort by time that shift was created
df = df.sort_values(by = 'sa_create').reset_index(drop = True)

In [14]:
# what are prevstatus column like when status = confirmed
df[df['status'] == 'confirmed']['prevStatus'].value_counts()

selected       42035
unavailable       11
applied            2
cancelled          1
Name: prevStatus, dtype: int64

In [15]:
# what are status column like when prevstatus = confirmed
df[df['prevStatus'] == 'confirmed']['status'].value_counts()

withdrawn    14953
cancelled     7334
applied          2
Name: status, dtype: int64

# Feature Engineering

### Target: urgent withdrawn as 1

In [16]:
def CW_by_nurse(row):
    if row['status']=='withdrawn' and row['prevStatus'] == 'confirmed':
        if row['withdrawninfo_value'] == 'nurse':
            if row['CW_Time2Start_Time'] > 0 and row['CW_Time2Start_Time'] <= 24:
                return 1
            else:
                return 0
        else:
            return 0
    else:
        return 0
    
df['target'] = df.apply (lambda row: CW_by_nurse(row), axis=1)

In [17]:
df['target'].value_counts()

0    172252
1      4629
Name: target, dtype: int64

###  Count previsous shift application

In [18]:
# track by using dictionary
count_prev_SA = []
test = df[['user_id', 'sa_create']]

# create SA dictionary, set all value = 0 -> dramatically reduce computational cost
uid_library = list(pd.unique(test['user_id']))
sa_dict = {} 
for uid in uid_library:
    sa_dict.update({uid: 0}) 
    
for i, v in enumerate(test['user_id']):
    sa_dict[v] += 1
    count_prev_SA.append(sa_dict[v]-1)
 
df['count_prev_SA'] = count_prev_SA

###  Count previsous urgent withdrawns

In [19]:
# track by using dictionary
count_prev_CW = []
test = df[['user_id', 'count_prev_SA', 'sa_create', 'target']]

# create cw dictionary, set all value = 0 -> dramatically reduce computational cost
uid_library = list(pd.unique(test['user_id']))
cw_dict = {} 
for uid in uid_library:
    cw_dict.update({uid: 0}) 
    
# fill dictionary and fill cw count
for i, v in enumerate(test['user_id']):
    if test['target'][i] == 1:
        cw_dict[v] += 1
        count_prev_CW.append(cw_dict[v]-1)
    else:
        count_prev_CW.append(cw_dict[v])
        
df['count_prev_CW'] = count_prev_CW

### Count previsous urgent withdrawns/Count previsous shift applications

In [20]:
df['prev_CW/SA_rate'] = df['count_prev_CW']/df['count_prev_SA']

# fill nan with 0, happend bc 0/0, meaning rate = 0
df['prev_CW/SA_rate'] = df['prev_CW/SA_rate'].fillna(0)

### Previous withdrawn times previous apply

In [21]:
df['prev_CW x SA_rate'] = df['count_prev_CW']*df['count_prev_SA']

### <font color = blue>keep only prevstatus or status = confirmed

In [22]:
df = df[df.apply(lambda row: (row['prevStatus'] == 'confirmed') or (row['status'] == 'confirmed'), axis = 1)].reset_index(drop = True)

In [23]:
# how many left
df.shape

(68583, 38)

### How many previsous urgent withdrawns in a month

In [25]:
import datetime
from datetime import datetime
from dateutil import relativedelta

# new column, convert start time to string
df['Str_Start_Time'] = df['Start_Time'].apply(lambda row: str(row))
# get only year, month, day in starttime
df['start_date'] = [w.split()[0] for w in list(df['Str_Start_Time'])]

# track by using dictionary, sort temp dataframe by userid then startdate
CW_in_a_month = []
df = df.sort_values(by=['user_id','start_date'], ascending = True).reset_index(drop = True)
test = df[['user_id', 'start_date', 'sa_create', 'target']].reset_index(drop = True)


for i, v in enumerate(test['user_id']):
    a = 0
    CW = 0
    delta = relativedelta.relativedelta(pd.to_datetime(test['start_date'][i-a]),
                                        pd.to_datetime(test['start_date'][i]))
    
    while delta.months > -1 and test['user_id'][i-a] == v: 
        if test['target'][i-a] == 1 and test['user_id'][i-a] == v and a >= 1:
            CW = 1
            break
    
        elif i-a <= 1:
            break

        elif test['user_id'][i-a] != v:
            break
            
        else:
            a += 1
            delta = relativedelta.relativedelta(pd.to_datetime(test['start_date'][i-a]),
                                                pd.to_datetime(test['start_date'][i]))
    CW_in_a_month.append(CW)

df['CW_in_a_month'] = CW_in_a_month

In [26]:
df['CW_in_a_month'].value_counts()

0    45599
1    22984
Name: CW_in_a_month, dtype: int64

### Find Extreme UCW rate fcaility

In [44]:
ucw_rate_list = []
# dict 1: all shift applications in this facilty
dict1 = df['f_name'].value_counts().to_dict()
# eliminate extreme situation by set number of visit >= 20
fac_over_20vis = [k for k, v in dict1.items() if v >= 20]

# dict 2: all ucw in this facility
dict2 = df[df['target'] == 1]['f_name'].value_counts().to_dict()

# create dict 3: ucw rate
ucw_rate = {} 
for fname in fac_over_20vis:
    ucw_rate.update({fname: 0}) 
# fill dict 3    
for key in fac_over_20vis:
    if key in list(dict2.keys()):
        ucw_rate[key] = (dict2[key]/dict1[key])

# top 7 highest ucw rate facilities name
highest_7_ucw_fac = [w[0] for w in sorted(ucw_rate.items(),key = lambda d:d[1], reverse = True)[:7]]
df['high_ucw_facilitites'] = df.apply(lambda row: 1 if row['f_name'] in highest_7_ucw_fac else 0, axis = 1)
# top 7 lowest ucw rate facilities name
lowest_7_ucw_fac = [w[0] for w in sorted(ucw_rate.items(),key = lambda d:d[1], reverse = True)[-7:]]
df['low_ucw_facilitites'] = df.apply(lambda row: 1 if row['f_name'] in lowest_7_ucw_fac else 0, axis = 1)

### Days of week

In [46]:
# monday -> 1, so on and so forth
df['days_of_week'] = df.apply(lambda row: row['Start_Time'].weekday() + 1, axis = 1)

### Start Time-time of the day

In [68]:
def get_part_of_day(hour):
    return (
        "morning" if 5 <= hour <= 11
        else
        "afternoon" if 12 <= hour < 17
        else
        "evening" if 18 <= hour <= 22
        else
        "night"
    )

df['Start_time_of_the_day'] = df.apply(lambda row: get_part_of_day(row['Start_Time'].hour), axis =1)

### <font color = red> csv for feature selection

In [72]:
df.to_csv('feature_selection.csv')

### <font color = blue> categorical

### 2 Type dummy: RN, LVN + LPN , Rest


In [26]:
df['type'].value_counts()

CNA             29116
STNA            18119
LVN              6349
RN               5685
LPN              3444
CMA/Med-Tech     3381
Tech             1042
Name: type, dtype: int64

In [27]:
df['type_RN'] = df.apply(lambda row:1 if row['type'] == 'RN' else 0, axis = 1)
df['type_LVN+LPN'] = df.apply(lambda row: 1 if row['type'] == 'LVN' or row['type'] == 'LPN' else 0, axis = 1)

### 1 SegementName dummy: Senior Living, Healthcare

In [28]:
df['segmentName'].value_counts()

Senior Living    59186
Healthcare        7950
Name: segmentName, dtype: int64

In [29]:
def create_segmentName_dummy(row):
    if row['segmentName']=='Senior Living':
        return 1
    else:
        return 0
    
df['segmentName_d'] = df.apply(lambda row: create_segmentName_dummy(row), axis=1)

### 5 Facility Area Name Dummy

In [30]:
df['areaName'].value_counts()

Houston           31420
Northeast Ohio    21806
DFW                9179
Austin             2766
San Antonio        1907
Cincinnati           58
Name: areaName, dtype: int64

In [31]:
# Houston
df['areaName_houston'] = df.apply(lambda row: 1 if row['areaName'] == 'Houston' else 0, axis = 1)

# Northeast Ohio
df['areaName_no'] = df.apply(lambda row: 1 if row['areaName'] == 'Northeast Ohio' else 0, axis = 1)

# DFW
df['areaName_dfw'] = df.apply(lambda row: 1 if row['areaName'] == 'DFW' else 0, axis = 1)

# Austin
df['areaName_austin'] = df.apply(lambda row: 1 if row['areaName'] == 'Austin' else 0, axis = 1)

# San Antonio
df['areaName_san'] = df.apply(lambda row: 1 if row['areaName'] == 'San Antonio' else 0, axis = 1)

### <font color = green>output feature dataframe

In [32]:
df.columns

Index(['id', 'user_id', 'shift_id', 'facility_id', 'withdrawninfo_value',
       'status', 'prevStatus', 'distance', 'shift_description',
       'assigned_nurse_id', 'net_pay', 's_unit', 'type', 's_qualifications',
       's_breakTime', 'withdrawnInfo', 'f_name', 'f_short_name', 'segmentName',
       'areaName', 's_create', 'sa_create', 'u_approve', 'u_create',
       'sa_statusupdate', 'Start_Time', 'U_create2U_approve', 'U_create2now',
       'U_approve2now', 'S_create2SA_Create', 'SA_Create2Start_Time',
       'S_Create2Start_Time', 'CW_Time2Start_Time', 'target', 'count_prev_SA',
       'count_prev_CW', 'prev_CW/SA_rate', 'prev_CW x SA_rate',
       'Str_Start_Time', 'start_date', 'CW_in_a_month', 'type_RN',
       'type_LVN+LPN', 'segmentName_d', 'areaName_houston', 'areaName_no',
       'areaName_dfw', 'areaName_austin', 'areaName_san'],
      dtype='object')

In [33]:
feature_df = df[['id', 'user_id', 'shift_id', 'prev_CW/SA_rate', 'status', 'S_create2SA_Create',
                 'S_Create2Start_Time','SA_Create2Start_Time','U_create2now', 'U_approve2now','prev_CW x SA_rate',
                 'type_RN', 'type_LVN+LPN', 'segmentName_d', 'areaName_houston', 'areaName_no', 'areaName_dfw', 
                 'areaName_austin', 'areaName_san', 'net_pay', 'target', 'sa_create', 'Start_Time',
                 'CW_in_a_month', 'count_prev_SA', 'count_prev_CW']]
feature_df.to_csv('model_data.csv')